<a href="https://colab.research.google.com/github/pradyot-09/DL-Reproducibility-Project/blob/master/distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
device = 'cuda'

In [0]:
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self, hidden_size=1200, dropout=0.0, hidden_dropout=0.0):
        super(Model, self).__init__()
        self.dropout = nn.Dropout(dropout)
        self.hidden1 = nn.Linear(784, hidden_size, bias=True)
        self.hidden1_dropout = nn.Dropout(hidden_dropout)
        self.hidden2 = nn.Linear(hidden_size, hidden_size, bias=True)
        self.hidden2_dropout = nn.Dropout(hidden_dropout)
        self.hidden3 = nn.Linear(hidden_size, 10, bias=True)

    def forward(self, x):
        x = self.dropout(x)
        x = F.relu(self.hidden1(x))
        x = self.hidden1_dropout(x)
        x = F.relu(self.hidden2(x))
        x = self.hidden2_dropout(x)
        return F.softmax(self.hidden3(x))

In [3]:
import torchvision
import torchvision.transforms as transforms
transform = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize((0.5,), (0.5,))
])

batch_size = 128

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


In [0]:
def convert_labels(labels):
    target = torch.zeros([len(labels), 10], dtype=torch.float32)
    for i, l in enumerate(labels):
      target[i][l] = 1.0
    return target

In [6]:
import torch.optim as optim

net = Model(dropout=0.2, hidden_dropout=0.5)
net.to(device)
criterion = torch.nn.MSELoss()
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = torch.optim.Adam(net.parameters(), lr=0.01, weight_decay=0.01)

for epoch in range(100):  # loop over the dataset multiple times
    running_loss = 0.0
    total = 0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = torch.flatten(inputs, start_dim=1).to(device)
        target = convert_labels(labels).to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        total += len(data)

        # print statistics
        running_loss += loss.item()
    # print every epoch
    print('[%d] loss: %.3f' % (epoch + 1, running_loss / total))

print('Finished Training')
torch.cuda.get_device_name(0)
PATH = './mnist_dropout_100_epoch.pth'
torch.save(net.state_dict(), PATH)
print('Saved Model')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[1] loss: 0.015
[2] loss: 0.008
[3] loss: 0.007
[4] loss: 0.006
[5] loss: 0.005
[6] loss: 0.005
[7] loss: 0.004
[8] loss: 0.004
[9] loss: 0.004
[10] loss: 0.003
[11] loss: 0.003
[12] loss: 0.003
[13] loss: 0.003
[14] loss: 0.003
[15] loss: 0.003
[16] loss: 0.003
[17] loss: 0.003
[18] loss: 0.003
[19] loss: 0.003
[20] loss: 0.002
[21] loss: 0.002
[22] loss: 0.002
[23] loss: 0.002
[24] loss: 0.002
[25] loss: 0.002
[26] loss: 0.002
[27] loss: 0.002
[28] loss: 0.002
[29] loss: 0.002
[30] loss: 0.002
[31] loss: 0.002
[32] loss: 0.002
[33] loss: 0.002
[34] loss: 0.002
[35] loss: 0.002
[36] loss: 0.002
[37] loss: 0.002
[38] loss: 0.002
[39] loss: 0.002
[40] loss: 0.002
[41] loss: 0.002
[42] loss: 0.002
[43] loss: 0.002
[44] loss: 0.002
[45] loss: 0.002
[46] loss: 0.002
[47] loss: 0.002
[48] loss: 0.002
[49] loss: 0.002
[50] loss: 0.002
[51] loss: 0.001
[52] loss: 0.001
[53] loss: 0.001
[54] loss: 0.001
[55] loss: 0.001
[56] loss: 0.001
[57] loss: 0.001
[58] loss: 0.001
[59] loss: 0.001
[60] l

In [0]:
torch.save(net.state_dict(), './mnist_dropout_100_epoch.pth')

In [7]:
net = Model().to(device)
net.load_state_dict(torch.load(PATH))

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs = torch.flatten(inputs, start_dim=1).to(device)
        target = convert_labels(labels).to(device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        _, target = torch.max(target.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

print('Accuracy of the network on the 10000 test images: %d %% (%d wrong out of %d)' % (
    100 * correct / total, total - correct, total))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy of the network on the 10000 test images: 98 % (127 wrong out of 10000)
